Solitaire - Deck of Cards Based Strong Encryption
============================
* * *
Introduction
----------------
This is an implementation of the Solitaire(aka. Pontifex) encryption system found in the novel *Cryptonomicon* written by Neal Stephensen. This cryptosystem was invented by Bruce Schneier, and this implementation is based on his description found in an appendix of the afformentioned novel. 

Solitaire is intended to use a standard deck of playing cards to generate a sequence of pseudo-random numbers that are moduarly added to the message to create the cyphertext. The receiver of the message can then moduarly subtract the same sequence from the cyphertext to recover the message, provided that the receiver has their deck in the same initial configuration as the sender. This initial configuration is the shared secret. 

Individual cards can be mapped to the numbers 1-52 in any way that is convenient. In Schneier's description, he suggests using the bridge ordering of suites(clubs, hearts, diamonds, spades) so that, for example, the ace of clubes is 1, the deuce of diamonds is 28, and so on. Whenever needed, the two jokers are both interpreted as 53. Also, the jokers must be somehow distinguishable. One of them will be referred to as joker 'A', the other as joker 'B'.

***

The steps to generate the keystream of pseudo-random numbers are as follows:
1. Find joker A. Move it one card down. If it is at the bottom of the deck, place it beneath the top card.
2. Find joker B. Move it two cards down. Again, if B is at the bottom of the deck, wrap around to the top. A and B should never be the top cards.
3. Do a triple cut of the deck based on the locations of the two jokers. For example, if the deck looks like
```
[Top cards]A[Middle cards]B[Bottom cards]
```
swap the top cards and bottom cards to make it look like:
```
[Bottom cards]A[Middle cards]B[Top cards]
```
If joker B happens to be nearer to the top, just swap A and B in the above diagram.
4. Look at the card on the bottom of the deck. Convert that card to a number and call it n. Now perform a cut after the top n cards, leaving the bottom card at the bottom. For example, if the deck looks like
```
[Top n cards][Remainder of deck][Bottom card]
```
Then after the cut it should look like
```
[Remainder of deck][Top n cards][Bottom card]
```
5. Convert the top card to a number, and look at the card at that position in the deck. Convert that card to a number. This number is your output key. If the card happens to be a joker, sorry, start again from step 1. Note that the deck is unchanged by this step.
6. **Repeat** steps 1-5 to generate as many keys as needed.

In [1]:
import random
from random import shuffle
#random.seed(0) #enable for testing

In [2]:
def keystream(deck):
    deck = deck[:] #make local copy of deck
    while True:
        #step 1:
        iA = deck.index('A')
        deck.pop(iA)
        iA = iA
        if iA == 53:
            iA = 1
        else:
            iA = iA+1
        deck.insert(iA,'A')
        #step 2
        iB = deck.index('B')
        deck.pop(iB)
        if iB == 53:
            iB = 2
        elif iB == 52:
            iB = 1
        else:
            iB = iB + 2
        deck.insert(iB,'B')
        #step 3
        i1 = min(iA, iB)
        i2 = max(iA, iB)
        deck = deck[i2+1:] + deck[i1:i2+1] + deck[:i1]
        #step 4
        bottom = deck[-1]
        n = bottom if type(bottom) is int else 53
        deck = deck[n:-1] + deck[:n] + [bottom,]
        #step 5
        n = deck[0] if type(deck[0]) is int else 53
        out = deck[n-1]
        if type(out) is int:
            yield out+1

To encode or decode the messages, just convert the input python strings to their appropriate numbers sequences eg. (```'aab'``` becomes ```0,0,1```). Then either add or subtract the number from the keystream.

In [3]:
def encode(message, deck):
    ks = keystream(deck)
    cypher = []
    if type(message) is str:
        message = message.lower().encode('ascii')
    for l in message:
        x = next(ks)
        cypher.append((l - 97 + x) % 26 + 97)
    cypher = bytes(cypher).decode('ascii')
    return cypher

def decode(cypher, deck):
    ks = keystream(deck)
    message = []
    if type(cypher) is str:
        cypher = cypher.lower().encode('ascii')
    for l in cypher:
        message.append((l - 97 - next(ks)) % 26 + 97)
    message = bytes(message).decode('ascii')
    return message
    

Prepare the deck

In [4]:
deck = []
for i in range(52):
    deck.append(i+1)
deck.extend(['A','B'])
shuffle(deck)

Let's have a look at some sample output from the keystream

In [5]:
ks = keystream(deck)
keys = [next(ks) for _ in range(20)]
print(keys)

[16, 11, 19, 14, 49, 8, 13, 32, 14, 47, 6, 35, 38, 13, 31, 22, 46, 37, 52, 10]


Now encrypt a short message and look at the cyphertext and the decrypt. Note that the message can only contain lowercase letters. No numbers, no spaces, no punctuation. 

In [6]:
message = 'thisisasecretmessagestoppleasedonttellanyonestop'
print('message:    ' + message)
cyphertext = encode(message, deck)
print('cyphertext: ' + cyphertext)
decrypt = decode(cyphertext, deck)
print('decrypt:    ' + decrypt)    

message:    thisisasecretmessagestoppleasedonttellanyonestop
cyphertext: jsbgfanysxxnfzjomlgoasrvqkjzovnqdrwfvcriuebhtkwd
decrypt:    thisisasecretmessagestoppleasedonttellanyonestop


Conclusion
-----------
As a computer based encryption scheme, Solitaire isn't all that useful. However, it is honest to goodness strong encryption that only requires a deck of cards and some patience. 